In [7]:
import pandas as pd
import numpy as np
import time
import warnings
import duckdb
from bs4 import BeautifulSoup
import requests
import unicodedata

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

year = 2025

In [8]:
def remove_accents(text):
    if not isinstance(text, str):
        return text
    # Normalize to NFKD (decomposes characters)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignore non-ASCII characters, then decode back to str
    return text.encode('ascii', 'ignore').decode('ascii')

df_teams = pd.read_csv("../src/team_info_xref.csv")
teams = df_teams.ABV.tolist()

df = pd.DataFrame()

for team in teams:
    time.sleep(6)
    html = requests.get(f"https://www.basketball-reference.com/teams/{team}/{year + 1}.html").text
    soup = BeautifulSoup(html, "lxml")
    target_ids = ["roster"]
    target_tables = [soup.find("table", {"id": tid}) for tid in target_ids]
    df_temp = [pd.read_html(str(tbl))[0] for tbl in target_tables][0]
    df_temp['Team'] = team
    display(df_temp)
    df = pd.concat([df, df_temp])
df = df[['Team', 'Player', 'Pos']]
df["Player"] = df["Player"].apply(lambda x: x.encode("latin1").decode("utf-8"))
df['Player'] = df['Player'].apply(remove_accents)
df['Player'] = df.Player.str.replace(" (TW)", "")
df.to_csv(f'../tables/{year}/plyr_pos_xref.csv', index=False)

,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,5.0,Dyson Daniels,SG,6-7,199,"March 17, 2003",au AU,3,NaN,ATL
1,7.0,Nickeil Alexander-Walker,SG,6-5,205,"September 2, 1998",ca CA,6,Virginia Tech,ATL
2,17.0,Onyeka Okongwu,C,6-10,240,"December 11, 2000",us US,5,USC,ATL
3,18.0,Mouhamed Gueye,PF,6-11,210,"November 9, 2002",sn SN,2,Washington State,ATL
4,10.0,Zaccharie Risacher,SF,6-8,200,"April 8, 2005",es ES,1,NaN,ATL
5,1.0,Jalen Johnson,SF,6-8,219,"December 18, 2001",us US,4,Duke,ATL
6,27.0,Vit Krejci,PG,6-8,195,"June 19, 2000",cz CZ,4,NaN,ATL
7,3.0,Luke Kennard,SG,6-5,206,"June 24, 1996",us US,8,Duke,ATL
8,14.0,Asa Newell,PF,6-10,220,"October 5, 2005",us US,R,Georgia,ATL
9,8.0,Kristaps PorziÅÄ£is,C,7-2,240,"August 2, 1995",lv LV,9,NaN,ATL


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,11.0,Payton Pritchard,PG,6-1,195,"January 28, 1998",us US,5,Oregon,BOS
1,4.0,Anfernee Simons,SG,6-3,200,"June 8, 1999",us US,7,NaN,BOS
2,30.0,Sam Hauser,PF,6-7,217,"December 8, 1997",us US,4,"Marquette, Virginia",BOS
3,9.0,Derrick White,SG,6-4,190,"July 2, 1994",us US,8,"Colorado-Colorado Springs, Colorado",BOS
4,7.0,Jaylen Brown,SF,6-6,223,"October 24, 1996",us US,9,California,BOS
5,88.0,Neemias Queta,C,7-0,248,"July 13, 1999",pt PT,4,Utah State University,BOS
6,55.0,Baylor Scheierman,SG,6-6,205,"September 26, 2000",us US,1,"South Dakota State, Creighton",BOS
7,27.0,Jordan Walsh,PF,6-6,205,"March 3, 2004",us US,2,Arkansas,BOS
8,28.0,Hugo GonzÃ¡lez,SF,6-6,200,"February 5, 2006",es ES,R,NaN,BOS
9,8.0,Josh Minott,PF,6-8,205,"November 25, 2002",us US,3,Memphis,BOS


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,21.0,Noah Clowney,PF,6-10,210,"July 14, 2004",us US,2,Alabama,BRK
1,20.0,Day'Ron Sharpe,C,6-10,265,"November 6, 2001",us US,4,UNC,BRK
2,33.0,Nic Claxton,C,6-11,215,"April 17, 1999",us US,6,Georgia,BRK
3,8.0,Egor DÑmin,PG,6-8,200,"March 3, 2006",ru RU,R,BYU,BRK
4,14.0,Terance Mann,SG,6-6,215,"October 18, 1996",us US,6,Florida State,BRK
5,1.0,Ziaire Williams,SF,6-9,185,"September 12, 2001",us US,4,Stanford,BRK
6,13.0,Tyrese Martin,SG,6-6,215,"March 7, 1999",us US,2,"Rhode Island, UConn",BRK
7,17.0,Michael Porter Jr.,SF,6-10,218,"June 29, 1998",us US,6,Missouri,BRK
8,4.0,Drake Powell,SG,6-5,195,"September 8, 2005",us US,R,UNC,BRK
9,22.0,Jalen Wilson,PF,6-6,220,"November 4, 2000",us US,2,Kansas,BRK


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,4.0,Sion James,SG,6-5,220,"December 4, 2002",us US,R,"Tulane, Duke",CHO
1,0.0,Miles Bridges,PF,6-7,225,"March 21, 1998",us US,6,Michigan State,CHO
2,7.0,Kon Knueppel,SF,6-6,215,"August 3, 2005",us US,R,Duke,CHO
3,14.0,Moussa DiabatÃ©,C,6-10,210,"January 21, 2002",fr FR,3,Michigan,CHO
4,8.0,Collin Sexton,SG,6-3,190,"January 4, 1999",us US,7,Alabama,CHO
5,1.0,LaMelo Ball,PG,6-7,180,"August 22, 2001",us US,5,NaN,CHO
6,11.0,Ryan Kalkbrenner,C,7-1,256,"January 17, 2002",us US,R,Creighton,CHO
7,33.0,Liam McNeeley,SF,6-7,210,"October 10, 2005",us US,R,UConn,CHO
8,23.0,Tre Mann,PG,6-4,178,"February 3, 2001",us US,4,Florida,CHO
9,31.0,Tidjane SalaÃ¼n,PF,6-10,207,"August 10, 2005",fr FR,1,NaN,CHO


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,14,Matas Buzelis,PF,6-8,209,"October 13, 2004",us US,1,NaN,CHI
1,9,Nikola VuÄeviÄ,C,6-9,260,"October 24, 1990",ch CH,14,USC,CHI
2,44,Patrick Williams,PF,6-6,215,"August 26, 2001",us US,5,Florida State,CHI
3,13,Kevin Huerter,SF,6-6,198,"August 27, 1998",us US,7,Maryland,CHI
4,11,Ayo Dosunmu,SG,6-4,200,"January 17, 2000",us US,4,Illinois,CHI
5,30,Tre Jones,PG,6-1,185,"January 8, 2000",us US,5,Duke,CHI
6,3,Josh Giddey,PG,6-7,216,"October 10, 2002",au AU,4,NaN,CHI
7,25,Jalen Smith,C,6-8,215,"March 16, 2000",us US,5,Maryland,CHI
8,35,Isaac Okoro,SG,6-4,225,"January 26, 2001",us US,5,Auburn,CHI
9,15,Julian Phillips,SF,6-6,198,"November 5, 2003",us US,2,Tennessee,CHI


...

,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,10.0,DeMar DeRozan,PF,6-6,220,"August 7, 1989",us US,16,USC,SAC
1,18.0,Russell Westbrook,SF,6-4,200,"November 12, 1988",us US,17,UCLA,SAC
2,23.0,Keon Ellis,SG,6-4,175,"January 8, 2000",us US,3,"Florida SouthWestern State College, Alabama",SAC
3,5.0,Nique Clifford,SG,6-5,175,"February 9, 2002",us US,R,"Colorado, Colorado State",SAC
4,17.0,Dennis SchrÃ¶der,PG,6-1,175,"September 15, 1993",de DE,12,NaN,SAC
5,9.0,Precious Achiuwa,C,6-8,243,"September 19, 1999",ng NG,5,Memphis,SAC
6,0.0,Malik Monk,SG,6-3,200,"February 4, 1998",us US,8,Kentucky,SAC
7,42.0,Maxime Raynaud,C,7-1,250,"April 7, 2003",fr FR,R,Stanford,SAC
8,19.0,Drew Eubanks,C,6-10,245,"February 1, 1997",us US,7,Oregon State,SAC
9,8.0,Zach LaVine,SG,6-5,200,"March 10, 1995",us US,11,UCLA,SAC


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,3.0,Keldon Johnson,SF,6-6,220,"October 11, 1999",us US,6,Kentucky,SAS
1,40.0,Harrison Barnes,PF,6-7,225,"May 30, 1992",us US,13,UNC,SAS
2,30.0,Julian Champagnie,SF,6-7,217,"June 29, 2001",us US,3,St. John's,SAS
3,24.0,Devin Vassell,SG,6-5,200,"August 23, 2000",us US,5,Florida State,SAS
4,7.0,Luke Kornet,C,7-1,250,"July 15, 1995",us US,8,Vanderbilt,SAS
5,11.0,Carter Bryant,PF,6-6,220,"November 26, 2005",us US,R,Arizona,SAS
6,4.0,De'Aaron Fox,PG,6-3,185,"December 20, 1997",us US,8,Kentucky,SAS
7,5.0,Stephon Castle,PG,6-6,215,"November 1, 2004",us US,1,UConn,SAS
8,2.0,Dylan Harper,SG,6-5,215,"March 2, 2006",us US,R,Rutgers University,SAS
9,1.0,Victor Wembanyama,C,7-4,235,"January 4, 2004",fr FR,2,NaN,SAS


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,3,Brandon Ingram,SF,6-8,190,"September 2, 1997",us US,9,Duke,TOR
1,4,Scottie Barnes,PF,6-8,237,"August 1, 2001",us US,4,Florida State,TOR
2,23,Jamal Shead,PG,6-1,200,"June 24, 2002",us US,1,Houston,TOR
3,1,Gradey Dick,SG,6-7,200,"November 20, 2003",us US,2,Kansas,TOR
4,5,Immanuel Quickley,PG,6-2,190,"June 17, 1999",us US,5,Kentucky,TOR
5,54,Sandro Mamukelashvili,C,6-9,240,"May 23, 1999",us US,4,Seton Hall,TOR
6,14,Ja'Kobe Walter,SG,6-4,180,"September 4, 2004",us US,1,Baylor,TOR
7,12,Collin Murray-Boyles,PF,6-7,245,"June 10, 2005",us US,R,South Carolina,TOR
8,30,Ochai Agbaji,SG,6-5,215,"April 20, 2000",us US,3,Kansas,TOR
9,77,Jamison Battle,SF,6-7,220,"May 10, 2001",us US,1,"George Washington, Minnesota, Ohio State",TOR


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,3.0,Keyonte George,PG,6-4,185,"November 8, 2003",us US,2,Baylor,UTA
1,28.0,Brice Sensabaugh,SF,6-6,235,"October 30, 2003",us US,2,Ohio State,UTA
2,22.0,Kyle Filipowski,C,6-11,250,"November 7, 2003",us US,1,Duke,UTA
3,10.0,Svi Mykhailiuk,SF,6-7,205,"June 10, 1997",ua UA,7,Kansas,UTA
4,13.0,Walter Clayton,PG,6-4,195,"March 6, 2003",us US,R,"Iona College, Florida",UTA
5,23.0,Lauri Markkanen,PF,7-1,240,"May 22, 1997",fi FI,8,Arizona,UTA
6,30.0,Jusuf NurkiÄ,C,6-11,290,"August 23, 1994",ba BA,11,NaN,UTA
7,19.0,Ace Bailey,SF,6-9,200,"August 13, 2006",us US,R,Rutgers University,UTA
8,8.0,Isaiah Collier,PG,6-4,210,"October 8, 2004",us US,1,USC,UTA
9,0.0,Taylor Hendricks,PF,6-9,215,"November 22, 2003",us US,2,Central Florida,UTA


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,7,Bub Carrington,PG,6-4,190,"July 21, 2005",us US,1,Pitt,WAS
1,3,CJ McCollum,SG,6-3,190,"September 19, 1991",us US,12,Lehigh,WAS
2,9,Justin Champagnie,SF,6-6,206,"June 29, 2001",us US,4,Pitt,WAS
3,35,Marvin Bagley III,PF,6-10,235,"March 14, 1999",us US,7,Duke,WAS
4,12,Tre Johnson,SG,6-5,190,"March 7, 2006",us US,R,Texas,WAS
5,27,Will Riley,SF,6-9,180,"February 10, 2006",ca CA,R,Illinois,WAS
6,20,Alex Sarr,C,7-0,205,"April 26, 2005",fr FR,1,NaN,WAS
7,18,Kyshawn George,SF,6-8,200,"December 12, 2003",ch CH,1,Miami (FL),WAS
8,0,Tristan Vukcevic (TW),C,7-0,220,"March 11, 2003",it IT,2,NaN,WAS
9,22,Khris Middleton,SF,6-7,222,"August 12, 1991",us US,13,Texas A&M,WAS


# Update NULL Positions

In [9]:
df = pd.read_csv(f"../tables/{year}/season_gamelogs.csv")
print('Null Pos Count:', df[(df.Pos.isnull())].shape[0])
df2 = pd.read_csv(f'../tables/{year}/plyr_pos_xref.csv')
orig_cols = df.columns
df = pd.merge(df, df2, how = 'left', on = ['Team', 'Player'], suffixes=('', '_y'))
df['Pos'] = df.Pos.fillna(df.Pos_y)
print('Null Pos Count:', df[(df.Pos.isnull())].shape[0])
df = df[orig_cols]
df.to_csv(f"../tables/{year}/season_gamelogs.csv", index=False)

Null Pos Count: 1579
Null Pos Count: 209
